<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller:  Recomendando el Blog de  Hernán Casciari 


[Hernán Casciari](https://hernancasciari.com/#bio), es un escritor argentino, que escribe blog posts con cuentos e historias  relacionadas con el futbol, su vida, infancia, y relaciones familiares con toques de ficción. Este [blog](https://hernancasciari.com/blog/) es  tan interesantes que en 2005 fue premiado como “El mejor blog del mundo” por Deutsche Welle de Alemania. 

El objetivo de este caso-taller es construir un sistema de recomendación basado en los contenidos de los posts utilizando similitud de las palabras usadas o temas de los cuentos.

## Instrucciones generales

1. Para desarrollar el *cuaderno* primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "Revisión por el compañero."

4. El archivo entregado debe poder ser ejecutado localmente por los pares. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos  en la misma ruta de acceso del cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos 

En la carpeta `data` se encuentran el archivo `blog_casciari.csv` con el título, la fecha de publicación, y el contenido de los cuentos publicados en el blog  de sr. Casciari. Cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.
   

In [118]:
# Libreria
import pandas as pd
import unidecode
import re
import spacy
nlp = spacy.load("es_core_news_sm")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

In [120]:
# Utilice este espacio para escribir el código.
file_path = 'blog_casciari.csv'
df = pd.read_csv(file_path)
df_info = df.info()
print(df_info)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  520 non-null    object
 1   fecha   520 non-null    object
 2   cuento  520 non-null    object
dtypes: object(3)
memory usage: 12.3+ KB
None


,titulo,fecha,cuento
0,El rincón blanco,1/11/08,De pronto yo estaba en el hogar donde pasé la ...
1,Mínimos avances en la cama,1/24/08,"Menos la cama, todo ha mejorado en este mundo...."
2,Don Marcos,2/19/08,"Dos veces, y no una, mi abuelo materno me ayud..."
3,Los dos rulfos,3/26/08,"A su regreso de México, mi amigo Comequechu no..."
4,La noticia no es el perro,4/15/08,"De repente, un video de You Tube recibe un mil..."


* Se realiza el cargue de la base, se puede cambiar el **file_path** para que coloque la direccion en donde reposa la base *datos_clientes.csv*
* La base cuenta con Titulo, fecha de publicación y cuento. contiene 520 registros (ningún nulo), todas sus variables son de tipo "object" y todos sus cuentos son solo del 2018

### 2. Homogenización de textos

Para cumplir con el objetivo de generar recomendaciones en esta sección debe preparar los posts para poder ser utilizados en su sistema de recomendación. Para ello, "limpie" y "tokenize" cada uno de los cuentos, describiendo detalladamente los pasos que realizo y si transformó o eliminó ciertas palabras. Para asistirlo en la tarea he creado listas de *stopwords* que están disponibles en la carpeta `data`. En su procedimiento ilustre la limpieza con el cuento 'La venganza del metegol'. (En su limpieza recuerde que el objetivo es generar recomendaciones a partir de la similitud de las palabras o temas de los cuentos)

In [123]:
# Utilice este espacio para escribir el código.
file_path_stopwords = 'extra_stopwords.csv'
extra_stopwords = pd.read_csv(file_path_stopwords, sep=',',header=None)
extra_stopwords.columns = ['stopwords']
extra_stopwords=set(extra_stopwords['stopwords'].to_list())
nlp.Defaults.stop_words |= extra_stopwords
# out = unidecode.unidecode(df)
# out = re.sub("[^\\w\\s]|\n", ' ', out)
# out = re.sub("\d+", "", out)

def text_cleaning(txt):
    
    # Eliminar caracteres especiales
    out = unidecode.unidecode(txt)
    out = re.sub("[^\\w\\s]|\n", ' ', out)
    out = re.sub("\d+", "", out)
    # Poner en minúsculas
    out = out.lower()
    # out = re.sub('tv', ' ', out)
    # out = re.sub('miniserie', 'miniseriedetv', out)
    out = re.sub('\s+', ' ', out)
    
    #NLP object
    out = nlp(out)
    # Eliminar Stopwords
    out = [token.text for token in out if not token.is_stop]
    out = " ".join(out)
    # Obtener los lemas de cada palabra
    lemmas =[token.lemma_ for token in nlp(out)]
    # Convertir la lista de lemmas nuevamente a texto
    out = " ".join(lemmas)
    # Remover palabras muy cortas
    out = [token.text for token in nlp(out) if len(token) > 2]
    out = " ".join(out)
    
    return out

In [125]:
df['cuento_cleaned'] = df['cuento'].apply(text_cleaning)
df

,titulo,fecha,cuento,cuento_cleaned
0,El rincón blanco,1/11/08,De pronto yo estaba en el hogar donde pasé la ...,hogar pasar infancia saber nariz ojo acostumbr...
1,Mínimos avances en la cama,1/24/08,"Menos la cama, todo ha mejorado en este mundo....",cama mejorado mundo cocinabar sopa fuego lenar...
2,Don Marcos,2/19/08,"Dos veces, y no una, mi abuelo materno me ayud...",abuelo materno ayudo escritor intencion conver...
3,Los dos rulfos,3/26/08,"A su regreso de México, mi amigo Comequechu no...",regreso mexico amigo comequechu conto historia...
4,La noticia no es el perro,4/15/08,"De repente, un video de You Tube recibe un mil...",video you tube recibir millon visita autoro go...
...,...,...,...,...
515,Instrucciones para la masturbación del hijo,8/18/07,Si lees estas líneas es porque hoy cumples tre...,lees linea cumpl trece ano muerto redacto bata...
516,El colmo de un campesino,9/24/07,"Hace algunos días Natalia Méndez, una editora ...",natalia mendez editorar libro infantil soler l...
517,La decadencia del Hombre Corbata,10/16/07,El actual Hombre Corbata es el último eslabón ...,actual hombre corbato eslabon hombre disfrazad...
518,El sentido del olfato en los trenes,10/26/07,Mi nombre no importa; no voy a presentarme. Lo...,nombre importar presentarme importar cara apar...


In [135]:
cuento_metegol = df[df['titulo'] == 'La venganza del metegol']
cuento_metegol

,titulo,fecha,cuento,cuento_cleaned
160,La venganza del metegol,11/17/15,El mes pasado me invitaron a presentar un libr...,mes invitar presentar libro aires libro futbol...


1. Llamamos la base con los stopwords adicionales y las agregamos a las que tenemos en nuestro modelo de spacy
   
2. Luego creamos una funcion que realiza las siguientes tareas:
- Se eliminan los caracteres especiales y las secuencias de dígitos numéricos.
- Convertimos todo el texto a minúsculas.
- Eliminamos los espacios adicionales que pueden haber quedado tras la eliminación de caracteres especiales.
- Convertimos el texto en un objeto de procesamiento de lenguaje natural utilizando un modelo de spaCy.
- Se eliminan todas las stopwords, tanto las incluidas por defecto en el modelo de spaCy como las que añadimos manualmente.
- Se obtienen los lemas de las palabras, es decir, sus formas básicas o raíces. Por ejemplo, la palabra "jugando" se convierte en "jugar".
- Eliminamos las palabras muy cortas (de dos caracteres o menos), que suelen ser irrelevantes en el análisis de texto.

Para probar la función hacemos un filtro para el cuento llamado *La venganza del metegol* e ilustramos con es el cuento original y luego como se muestra al correr la función descrita anteriormente. Al usar la función vemos que el resultado es una lista con la limpieza y la tokenización del cuento.

### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones de cuentos en el blog a un usuario que leyó 'La venganza del metegol'. Para ello vamos a utilizar distintas estrategias.

#### 3.1. Recomendaciones basadas en contenidos

##### 3.1.1. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando en la distancia de coseno donde el texto este vectorizado por `CountVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones.

In [127]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cuento_cleaned'])

idx_metegol = df[df['titulo'] == 'La venganza del metegol'].index[0]

cos_sim = cosine_similarity(X[idx_metegol], X)

similarity_scores = list(enumerate(cos_sim[0]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

top_5_recommendations = [df['titulo'][i] for i, _ in similarity_scores[1:6]]
print("Top 5 recomendaciones:")
for i, title in enumerate(top_5_recommendations, 1):
    print(f"{i}. {title}")


Top 5 recomendaciones:
1. Cuento con bruja y tramontina
2. Una línea de puntos en un libro de catecismo
3. El milagro de los pueblos
4. Dice el Chiri, dice el Gordo
5. Hace seis años también era domingo


Para realizar la recomendación usamos *CountVectorizer* para convertir los cuentos en vectores de términos, depués se identifica la posición del cuento *La venganza del metegol* ya con esto se calcula la distancio del coseno entre este cuento y el resto, se ordena de mayor a menor por similitud y se toman solo las 5 primeras recomendaciones.

##### 3.1.2. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para  el cuento 'La venganza del metegol' usando nuevamente la distancia de coseno, pero ahora vectorice el texto usando `TF-IDFVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados del punto anterior y explique sus similitudes y/o diferencias.

In [128]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['cuento_cleaned'])

idx_metegol = df[df['titulo'] == 'La venganza del metegol'].index[0]

cos_sim_tfidf = cosine_similarity(X_tfidf[idx_metegol], X_tfidf)

similarity_scores_tfidf = list(enumerate(cos_sim_tfidf[0]))
similarity_scores_tfidf = sorted(similarity_scores_tfidf, key=lambda x: x[1], reverse=True)

top_5_recommendations_tfidf = [df['titulo'][i] for i, _ in similarity_scores_tfidf[1:6]]
print("Top 5 recomendaciones usando TF-IDF:")
for i, title in enumerate(top_5_recommendations_tfidf, 1):
    print(f"{i}. {title}")


Top 5 recomendaciones usando TF-IDF:
1. Una línea de puntos en un libro de catecismo
2. Cuento con bruja y tramontina
3. Dice el Chiri, dice el Gordo
4. Abrir y cerrar un círculo
5. La foto de Wasmosy


Hacemos la transformación de la variable con el texto limpio y tokenizado, en este caso usamos TfidfVectorizer. Identificamos el indice de *La venganza del metegol*, con esto calculamos la similitud del coseno y ordenamos de mayor a menor esta distancia. Ya con esto identificamos cuales son los 5 cuentos con mayor similitud a *La venganza del metegol*.

Se observa que 3 de los cuentos se mantienen en el top 5. Si uno realiza la lectura de lo cuentos.

##### 3.1.3. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando el texto vectorizado por `TF-IDFVectorizer` y la correlación como medida de similitud. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados de los puntos anteriores y explique sus similitudes y/o diferencias.

In [129]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['cuento_cleaned'])

idx_metegol = df[df['titulo'] == 'La venganza del metegol'].index[0]

X_tfidf_array = X_tfidf.toarray()

# Calcular la matriz de correlación entre todos los cuentos
correlation_matrix = np.corrcoef(X_tfidf_array)

# Extraer la fila correspondiente a "La venganza del metegol"
correlation_metegol = correlation_matrix[idx_metegol]

# Ordenar los cuentos según la correlación (de mayor a menor)
similarity_scores_correlation = list(enumerate(correlation_metegol))
similarity_scores_correlation = sorted(similarity_scores_correlation, key=lambda x: x[1], reverse=True)

# Obtener los 5 cuentos más recomendados (excluyendo el propio cuento)
top_5_recommendations_correlation = [df['titulo'][i] for i, _ in similarity_scores_correlation[1:6]]

# Mostrar las recomendaciones
print("Top 5 recomendaciones usando correlación:")
for i, title in enumerate(top_5_recommendations_correlation, 1):
    print(f"{i}. {title}")


Top 5 recomendaciones usando correlación:
1. Una línea de puntos en un libro de catecismo
2. Cuento con bruja y tramontina
3. Dice el Chiri, dice el Gordo
4. La foto de Wasmosy
5. Los jefes y los empleados


El procedimiento en está ocasión es el mismo del anterior, lo único que cambia, es la metrica que se usa para identificar cuales son los textos más similares a *La venganza del metegol*, en este caso hacemos uso de la correlación. Los resultados evidencian que con respecto al punto anterior, hay 4 cuentos similares que inclusiven mantienen el mismo orden y respecto al *CountVectorizer*  se mantienen 3 de estos. 

##### 3.2. Recomendaciones basadas en temas

Usando modelado de temas con LDA, encuentre los temas subyacentes en el blog. Explique como eligió el numero óptimo de temas. Utilizando el tema asignado al cuento 'La venganza del metegol' y la probabilidad de pertenecer a este tema genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para este cuento. Explique el procedimiento que realizó. Compare con los resultados encontrados anteriormente y explique sus similitudes y/o diferencias. (Esto puede tomar mucho tiempo y requerir mucha capacidad computacional, puede aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [141]:
df['cuento_cleaned_2'] = df['cuento_cleaned'].apply(lambda x: [token.text for token in nlp(x) if not token.is_stop])

dictionary = corpora.Dictionary(df['cuento_cleaned_2'])
corpus = [dictionary.doc2bow(text) for text in df['cuento_cleaned_2']]

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, random_state=42)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(dictionary, corpus, df['cuento_cleaned_2'], limit=10)
optimal_topics = model_list[coherence_values.index(max(coherence_values))]

for idx, topic in optimal_topics.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}")

lda_corpus = optimal_topics[corpus]

df['dominant_topic'] = [max(prob, key=lambda x: x[1])[0] for prob in lda_corpus]
df['topic_probability'] = [max(prob, key=lambda x: x[1])[1] for prob in lda_corpus]

idx_metegol = df[df['titulo'] == 'La venganza del metegol'].index[0]
metegol_topic = df.loc[idx_metegol, 'dominant_topic']

same_topic_cuentos = df[df['dominant_topic'] == metegol_topic].sort_values(by='topic_probability', ascending=False)

top_5_recommendations_lda = same_topic_cuentos['titulo'].iloc[1:6].values
print("Top 5 recomendaciones basadas en LDA:")
for i, title in enumerate(top_5_recommendations_lda, 1):
    print(f"{i}. {title}")

Topic: 0 
Words: 0.006*"ano" + 0.005*"cosa" + 0.005*"casa" + 0.003*"viejo" + 0.003*"pasar" + 0.003*"hernar" + 0.003*"casciari" + 0.003*"gente" + 0.003*"luca" + 0.003*"ojo"
Topic: 1 
Words: 0.008*"ano" + 0.004*"tiempo" + 0.004*"mundo" + 0.004*"casciari" + 0.003*"casa" + 0.003*"mirar" + 0.003*"pasar" + 0.003*"cosa" + 0.003*"vida" + 0.003*"argentino"
Topic: 2 
Words: 0.005*"ano" + 0.003*"casa" + 0.003*"mirar" + 0.003*"mundo" + 0.003*"padre" + 0.003*"cosa" + 0.003*"tiempo" + 0.003*"quedar" + 0.003*"dejar" + 0.003*"pasar"
Topic: 3 
Words: 0.006*"ano" + 0.005*"casa" + 0.004*"cosa" + 0.004*"pasar" + 0.004*"mundo" + 0.003*"hernar" + 0.003*"mirar" + 0.003*"casciari" + 0.003*"noche" + 0.003*"dejar"
Top 5 recomendaciones basadas en LDA:
1. La noticia no es el perro
2. El móvil de Hansel y Gretel
3. La trampa de McCracken
4. La revancha
5. Una cena demasiado larga


Para poder realizar el modelo LDA es necesario tener en un formato especial el insumo, lo primero que hacemos es poner en este formato la información que ya se limpio. Creamos el diccionario y la bolsa de palabras para el LDA, creamos una función para seleccionar el número optimo de temas y se ejecuta con el diccionario creado, la bolsa de palabras, la variable en el formato requerido y un limite de tomas de 10 entonces es función probará del 1 al 10 para tomar el número optimo de temas y con esto ejecutar el modelo. En este caso el numero de temas optimo fue 4, con esto obtenemos las probabilidades del tema para cada cuento, asignamos el tema dominante de cada cuento, obtenemos los cuentos que tienen el mismo tema dominante que *La venganza del metegol* y con esto generamos las 5 recomendaciones basadas en el tema.

En este caso los resultados no sé parecen en nada comparadas con el resto de las metodologias usadas, este top 5 de cuentos nunca se habían observado.

### 4 Recomendaciones generales

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para la entrada elegida? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

Muchos de los textos de Casciari son narraciones personales o anecdóticas que involucran situaciones cotidianas, se tocan temas de amistades y relaciones. Este caso puede haber un lenguaje muy similar en cada texto por lo que metodos basados en el uso de palabras pueden estar un tanto sesgados, por tal motivo, es mucho más valioso el analisis realizado por LDA pues este metodo clasifica los textos en grupos basados en temas latentes que pueden no ser obvios a partir de la similitud de palabras. Por eso mi recomendación sería usar el metodo LDA en esta caso, puede que en otros analisis los otros metodos arrojen mejores resultados y sean más eficientes.